In [1]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import load_metric
# !pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
import csv

# notebook_login()

CUDA_LAUNCH_BLOCKING=1


In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
train = pd.read_csv(r'train.csv', encoding='utf_8_sig')
test = pd.read_csv(r'test.csv', encoding='utf_8_sig')


In [4]:
train

,label,text,diff
0,negative,Fixed link errors,diff --git a/src/irisnet/CMakeLists.txt b/src/...
1,positive,Check types to avoid invalid reads/writes.,diff --git a/src/file.c b/src/file.c\nindex 4d...
2,positive,https://github.com/ImageMagick/ImageMagick/iss...,diff --git a/coders/png.c b/coders/png.c\ninde...
3,positive,XSS 취약점 수정,diff --git a/adm/boardgroup_form.php b/adm/boa...
4,negative,Change distribution URL.\n\ngit-svn-id: https:...,diff --git a/c/xml-security-c.spec b/c/xml-sec...
5,negative,Merge pull request #2426 from alvarobartt/deve...,diff --git a/nltk/tag/stanford.py b/nltk/tag/s...
6,negative,2021 license,diff --git a/Gruntfile.js b/Gruntfile.js\ninde...
7,positive,SPOOLSS: Try to avoid an infinite loop.\n\nUse...,diff --git a/epan/dissectors/packet-dcerpc-spo...
8,negative,Upgrade: Bump grape from 1.3.3 to 1.5.0\n\nBum...,diff --git a/Gemfile.lock b/Gemfile.lock\ninde...
9,negative,Merge branch '8.5' into 9.5,diff --git a/src/Framework/Assert.php b/src/Fr...


In [5]:
len(train)

10

In [6]:
train['label'].value_counts()

label
negative    6
positive    4
Name: count, dtype: int64

In [7]:
test['label'].value_counts()

label
negative    6341
positive    3761
Name: count, dtype: int64

In [8]:
# !pip install sentence_transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('../../sentence-transformers/paraphrase-mpnet-base-v2')

In [9]:
train['embeddings'] = train['text'].apply(lambda x: model.encode(str(x)))
test['embeddings'] = test['text'].apply(lambda x: model.encode(str(x)))

In [10]:
test

,label,text,diff,embeddings
0,negative,Copyright header.,diff --git a/src/main/java/org/projectforge/ad...,"[0.10010557, -0.13413383, -0.0052697547, 0.032..."
1,negative,:art: 部分类增加实现序列化接口,diff --git a/weixin-java-mp/src/main/java/me/c...,"[0.11699601, -0.064321205, -0.036785033, 0.024..."
2,negative,Update copyright to 2020,diff --git a/README.rst b/README.rst\nindex 76...,"[0.05565265, 0.37451336, -0.0034549485, -0.130..."
3,negative,test case for testing old knownnodes pickle\n-...,diff --git a/src/network/knownnodes.py b/src/n...,"[-0.080094546, 0.14758536, 0.027495546, 0.0664..."
4,negative,Introspection fixes.,diff --git a/gsf/gsf-docprop-vector.c b/gsf/gs...,"[-0.0144902775, 0.2714296, -0.027338658, 0.000..."
...,...,...,...,...
10097,negative,Fix alignment,diff --git a/wwwroot/cgi-bin/awstats.pl b/wwwr...,"[-0.12206936, -0.13066295, -0.1028094, 0.01713..."
10098,negative,hwdb: Chuwi Hi12 (#17042)\n\nDifferent bios ve...,diff --git a/hwdb.d/60-sensor.hwdb b/hwdb.d/60...,"[-0.062666506, -0.1289232, 0.048642106, -0.153..."
10099,negative,Clarify text regarding enabling/disabling modu...,diff --git a/config-templates/config.php b/con...,"[0.054796852, -0.23509139, -0.031717252, 0.119..."
10100,negative,Merge branch 'documentation-update' into 'mast...,diff --git a/docs/api/index.rst b/docs/api/ind...,"[-0.12838666, 0.07940822, -0.011249377, 0.0323..."


In [11]:
X_train = train['embeddings'].to_list()
X_test = test['embeddings'].to_list()

y_train = train['label'].to_list()
y_test = test['label'].to_list()

In [12]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(X_train,y_train)

LogisticRegression()

In [14]:
from sklearn import metrics
predicted = LR.predict(X_test)

print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted,average='weighted'))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted,average='weighted'))
print("Logistic Regression Recall:",metrics.f1_score(y_test, predicted,average='weighted'))
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))

Logistic Regression Precision: 0.6815428044124127
Logistic Regression Recall: 0.6899623836863987
Logistic Regression Recall: 0.6641527018666288
Logistic Regression Accuracy: 0.6899623836863987
